In [19]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import config
import os.path
from os import path

from create_db import create_connection, create_table, clean_column_names
from populate_db import extract_data, insert_records
import query_db as qd

import importlib

In [87]:
importlib.reload(qd)

<module 'query_db' from '/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/query_db.py'>

In [94]:
pd.set_option('display.max_columns', None)

## High level understanding of the datasets:

Tables needed to construct recidivism flag:
- Court commitment
- Sentence Computation
- Maybe sentence component

Tables needed to construct features (in addition to those listed above):
- Inmate profile
- Offender profile
- Disciplinary infractions

Below, I'm going to walk through the 3 tables needed to construct recidivism flag using an example offender, DOC number: 0130556. 

Some resources I used to understand this offender's history:
- [Blog post about this case](https://nccriminallaw.sog.unc.edu/running-sentence-consecutively-anticipated-sentence/)
- [Offender Search](https://webapps.doc.state.nc.us/opi/viewoffender.do?method=view&offenderID=0130556&searchLastName=FLEMING&searchFirstName=TIMOTHY&searchMiddleName=C&searchDOBRange=0&listurl=pagelistoffendersearchresults&listpage=1)
- [Structure of NC DOC database](https://www.doc.state.nc.us/offenders/PublicTables.pdf)


### Start off by looking at the full sentencing history of this individual in Sentence Component

In [88]:
query_sentence_component_example = '''
                                    SELECT * FROM OFNT3CE1 WHERE OFFENDER_NC_DOC_ID_NUMBER = '0130556'
                                    '''

In [89]:
conn = create_connection(config.database_name)
sentence_component_example = qd.query_db_notebook(conn,query_sentence_component_example)
conn.close

<function Connection.close>

In [90]:
sentence_component_example

,OFFENDER_NC_DOC_ID_NUMBER,COMMITMENT_PREFIX,SENTENCE_COMPONENT_NUMBER,COUNTY_OF_CONVICTION_CODE,COURT_DOCKET_NUMBER,PUNISHMENT_TYPE_CODE,COURT_TYPE_CODE,COMPONENT_DISPOSITION_CODE,CMP_DISPOSITION_CODE_2_OF_2,NUMBER_OF_COUNTS,TYPE_OF_COUNT_CODE,PRIMARY_OFFENSE_CODE,OFFENSE_QUALIFIER_CODE,DATE_OFFENSE_COMMITTED__BEGIN,DATE_OFFENSE_COMMITTED__END,NC_GENERAL_STATUTE_NUMBER,PRIMARY_FELONYMISDEMEANOR_CD,SENTENCING_PENALTY_CLASS_CODE,PRIOR_RCD_POINTSCONVICTIONS,PRIOR_RECORD_LEVEL_CODE,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,LENGTH_OF_SUPERVISION,SUPERVISION_TERM_EXTENSION,SUPERVISION_TO_FOLLOW_INCAR,SPLIT_SENTENCE_ACTIVE_TERM,GS_MAXIMUM_SENTENCE_ALLOWED,SERVING_MIN_OR_MAX_TERM_CODE,SENTENCE_TYPE_CODE,SENTENCE_TYPE_CODE1,SENTENCE_TYPE_CODE2,SENTENCE_TYPE_CODE3,SENTENCE_TYPE_CODE4,SENTENCE_TYPE_CODE5,CREDITS_FOR_JAIL_DAYS_SERVED,ICC_JAIL_CREDITS_IN_DAYS,SENTENCE_CHAINING_TYPE_CODE,PRIOR_COMMITMENT_PREFIX,PRIOR_COMPONENT_IDENTIFIER,PP_SUPVTERM_CHAIN_TYPE_CODE,PP_PRIOR_COMMITMENT_PREFIX,PRIOR_PP_COMMNTCOMPONENT_ID,TIME_COMPUTATION_EXCEPTION_CD,SENTENCE_CONVICTION_DATE,SENTENCE_EFFECTIVEBEGIN_DATE,DELEGATED_AUTHORITY_FLAG,INMATE_SENTENCE_STATUS_CODE,INMATE_COMPONENT_STATUS_DATE,PP_CASE_STATUS,PP_COMPONENT_STATUS_DATE,DATE_OF_LAST_UPDATE,TIME_OF_LAST_UPDATE,ORIGINAL_DATA_ENTRY_DATE,ORIGINAL_SENTENCE_AUDIT_CODE,DATE_OF_LAST_UPDATE_TWO,TIME_OF_LAST_UPDATE_TWO
0,0130556,01,001,NEW HANOVER,92020074,PRE-SS (FAIR) DCC,DISTRICT,GUILTY,GUILTY,001,None,DRUG PARA - USE/POSSESS,PRINCIPAL,1992-10-09,1992-10-09,None,MISD.,MISD.(PRE-STRUCTURE),000,None,0010000,0000000,None,None,None,0000000,0000000,None,PROBATION,SUSPENDED SENTENCE,COUNTY JAIL,None,None,None,00000,00000,None,None,None,INITIAL,None,None,NOT APPLICABLE,1992-10-23,1992-10-23,None,None,0001-01-01,ELECT TO SERVE SERVE,1995-02-07,0001-01-01,01:00:00,0001-01-01,None,0001-01-01,01:00:00
1,0130556,02,001,NEW HANOVER,94021895,PRE-SS (FAIR) DCC,SUPERIOR,GUILTY,GUILTY,001,None,LARCENY (OVER $200),PRINCIPAL,1994-07-11,1994-07-11,None,FELON,CLASS H,000,None,0030000,0000000,None,None,None,0000000,0000000,None,PROBATION,SUSPENDED SENTENCE,DEPT OF CORR DIV OF PRISONS,None,None,None,00000,00000,None,None,None,INITIAL,None,None,NOT APPLICABLE,1995-04-06,1995-04-06,None,None,0001-01-01,ELECT TO SERVE SERVE,1996-01-11,1996-01-17,10:04:55,0001-01-01,None,0001-01-01,01:00:00
2,0130556,03,001,MECKLENBURG,97020381,COMMUNITY SS (DCC),DISTRICT,GUILTY,None,001,CONCURRENT,DRIV LICENSE REVOKED,PRINCIPAL,1997-05-29,1997-05-29,20-28,MISD.,CLASS 1 MISDEMEANOR SS,000,LEVEL II,0000115,0000115,None,None,None,000 0 0,None,MAX.TERM:,PROBATION,SUSPENDED SENTENCE,COUNTY JAIL,None,None,None,00000,00000,None,None,None,INITIAL,None,None,NOT APPLICABLE,1997-08-01,1997-08-01,Y,None,0001-01-01,REVOKED REVOK,1998-07-08,1998-07-28,10:36:40,1997-08-27,None,1997-08-27,14:29:17
3,0130556,04,001,MECKLENBURG,97040899,COMMUNITY SS (DCC),SUPERIOR,GUILTY,None,001,CONCURRENT,USE SMOKE SCREEN,PRINCIPAL,1997-10-24,1997-10-24,90-95,FELON,CLASS I,003,LEVEL II,0000600,0000800,None,None,None,000 0 0,None,MAX.TERM:,PROBATION,None,None,None,None,None,00000,00000,None,None,None,CONCURRENT,03,001,NOT APPLICABLE,1998-05-01,1998-05-01,N,None,0001-01-01,REVOKED REVOK,1998-07-08,1998-07-28,10:36:40,1998-05-06,None,1998-05-06,07:41:56
4,0130556,AA,001,NEW HANOVER,92020074,FAIR MISDEMEAN,DISTRICT,UNKNOWN,None,001,None,DRUG PARA - USE/POSSESS,PRINCIPAL,0001-01-01,0001-01-01,None,MISD.,MISD.(PRE-STRUCTURE),000,None,0000000,0010000,0000000,0000000,0000502,0000000,0000000,MAX.TERM:,DEPT OF CORR DIV OF PRISONS,PROBATION REVOCATION,None,None,None,None,00001,00000,INITIAL,None,None,None,None,None,NOT APPLICABLE,1992-10-23,1995-02-07,None,ACTIVE,1995-02-10,None,0001-01-01,0001-01-01,01:00:00,0001-01-01,CONVERSION CS,0001-01-01,01:00:00
5,0130556,BA,001,NEW HANOVER,94021895,FAIR FELONS,SUPERIOR,UNKNOWN,None,001,CONCURRENT,LARCENY (OVER $200),PRINCIPAL,1994-07-11,1994-07-11,14-72,FELON,CLASS H,000,None,0000000,0030000,None,None,None,0000000,0000000,MAX.TERM:,

## Key takeaways and variables from sentence component table

- commitment prefixes that are numbers correspond to probation, commitment prefixes that are letters correspond to incarceration. See "Sentence Type Code" to confirm
- Within a given commitment prefix, there can be many sentence component numbers. This means that for the same concurrent sentence, an individual was convicted of / charged for multiple separate offenses. To confirm this, see note in blog post that says "In 2016 ... the defendant was convicted of common law robbery, conspiracy to commit common law robbery, and some other charges." Based on the offender search results and table above, I think this corresponds to commitment prefix BD
- For our purposes, I think it would make sense to consolidate sentence component numbers so that we only have 1 commitment prefix (reflecting each new sentence) and keep the "most serious offense" out of the multiple charges in a sentence
- Luckily, the court commitment table basically does that for us

### Next, look at Court Commitments for this individual

In [91]:
query_court_commitment_example = '''
                                    SELECT * FROM OFNT3BB1 WHERE OFFENDER_NC_DOC_ID_NUMBER = '0130556'
                                    '''

In [92]:
conn = create_connection(config.database_name)
court_commitment_example = qd.query_db_notebook(conn,query_court_commitment_example)
conn.close

<function Connection.close>

In [93]:
court_commitment_example

,OFFENDER_NC_DOC_ID_NUMBER,COMMITMENT_PREFIX,COMMITTED_LAST_NAME,COMMITTED_FIRST_NAME,COMMITTED_MIDDLE_NAME,COMMITTED_NAME_SUFFIX,OFFENDER_ADMISSIONINTAKE_DATE,PP_CASE_INTAKE_DATE,INMATE_COMMITMENT_STATUS_FLAG,COMMITMENT_STATUS_DATE,EARLIEST_SENTENCE_EFFECTIVE_DT,NEW_PERIOD_OF_INCARCERATION_FL,MOST_SERIOUS_OFFENSE_CODE,CO_OF_CONV_MOST_SERIOUS_OFFNSE,TOTAL_SENTENCE_LENGTH,TOTAL_JAIL_CREDITS_IN_DAYS,NO_RESTITUTION_FLAG,PP_COMMITMENT_STATUS_FLAG,PP_COMMITMENT_STATUS_DATE,TOTAL_LENGTH_OF_SUPERVISION,PED_PRIOR_TO_1995_CONVERSION,DATE_OF_LAST_UPDATE,TIME_OF_LAST_UPDATE,NEW_PERIOD_OF_SUPERVISION_FLAG,TYPE_OF_OLD_PE_DATE_CODE
0,0130556,01,FLEMING,TIMOTHY,CHADWICK,None,0001-01-01,1992-10-23,None,0001-01-01,1992-10-23,None,None,None,None,0000,None,ELECT TO SERVE SERVE,1995-02-07,None,0001-01-01,0001-01-01,01:00:00,None,None
1,0130556,02,FLEMING,TIMOTHY,CHADWICK,None,0001-01-01,1995-04-06,None,0001-01-01,1995-04-06,None,None,None,None,0000,None,ELECT TO SERVE SERVE,1996-01-11,None,0001-01-01,1996-01-17,10:04:55,None,None
2,0130556,03,FLEMING,TIMOTHY,CHADWICK,None,0001-01-01,1997-08-01,None,0001-01-01,1997-08-01,N,DRIV LICENSE REVOKED,MECKLENBURG,None,0000,None,REVOKED REVOK,1998-07-08,None,0001-01-01,1998-07-28,10:36:40,Y,None
3,0130556,04,FLEMING,TIMOTHY,CHADWICK,None,0001-01-01,1997-08-01,None,0001-01-01,1998-05-01,N,USE SMOKE SCREEN,MECKLENBURG,None,0000,None,REVOKED REVOK,1998-07-08,None,0001-01-01,1998-07-28,10:36:40,N,None
4,0130556,AA,FLEMING,TIMOTHY,CHAD,None,1995-02-10,1995-03-09,ACTIVE,1995-02-10,1995-02-07,Y,None,None,None,None,None,NORMAL NORM,1995-03-09,None,1995-02-06,0001-01-01,01:00:00,None,REG.PAROLE
5,0130556,BA,FLEMING,TIMOTHY,CHAD,None,1996-04-18,1996-08-01,ACTIVE,1996-04-18,1996-01-11,Y,LARCENY (OVER $200),NEW HANOVER,None,0000,None,NORMAL NORM,1996-08-01,None,0001-01-01,1996-08-22,15:21:08,N,None
6,0130556,BB,FLEMING,TIMOTHY,CHAD,None,2000-03-02,2006-05-17,ACTIVE,2000-03-02,2000-02-14,Y,HABITUAL FELON,MECKLENBURG,None,0151,None,NORMAL NORM,2006-05-17,None,0001-01-01,2006-05-17,17:13:57,Y,None
7,0130556,BC,FLEMING,TIMOTHY,CHAD,None,2009-11-19,0001-01-01,ACTIVE,2009-11-19,2009-10-23,Y,POSSESS WITS SCHEDULE I,MECKLENBURG,0010400,0098,None,NORMAL NORM,0001-01-01,0000000,0001-01-01,2011-03-08,14:38:36,N,None
8,0130556,BD,FLEMING,TIMOTHY,CHADWICK,None,2015-07-22,0001-01-01,ACTIVE,2015-07-22,2015-07-16,Y,HABITUAL FELON,MECKLENBURG,0280125,0774,None,NORMAL NORM,0001-01-01,0000000,0001-01-01,2016-07-14,09:23:57,N,None


## Key Takeaways and Variables from Court Commitment

- As we see above, the data is already "unique" on offender number and commitment prefix. I think the important variables here are:
    - New Period of incarceration flag (N or None for probation, Y for a new sentence resulting in incarceration)
    - most serious offense code (i assume of all the different charges in a sentence)
    - earliest sentence effective date

- Using this, we almost have everything we need to figure out if an individual recidivated, and if so, what the public safety risk associated with that recidivism is

- There are two caveats:
    - I don't see any "sentence end date" in this file. Luckily, Sentence Computation table has these
    - Some offense codes are missing, but we could get these ourselves from the Sentece Component File


### Finally, look at Sentence Computation for this individual

In [95]:
query_sentence_computation_example = '''
                                    SELECT * FROM INMT4BB1 WHERE INMATE_DOC_NUMBER = '0130556'
                                    '''

In [96]:
conn = create_connection(config.database_name)
sentence_computation_example = qd.query_db_notebook(conn,query_sentence_computation_example)
conn.close

<function Connection.close>

In [97]:
sentence_computation_example

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,INMATE_SENTENCE_COMPONENT,INMATE_COMPUTATION_STATUS_FLAG,SENTENCE_BEGIN_DATE_FOR_MAX,ACTUAL_SENTENCE_END_DATE,PROJECTED_RELEASE_DATE_PRD,PAROLE_DISCHARGE_DATE,PAROLE_SUPERVISION_BEGIN_DATE
0,0130556,AA,001,EXPIRED,1995-02-07,1995-03-09,1995-08-08,0001-01-01,0001-01-01
1,0130556,BA,001,EXPIRED,1996-01-11,1996-08-01,1997-06-17,1997-04-14,1996-08-01
2,0130556,BA,002,EXPIRED,1996-01-11,1996-07-19,1996-07-19,0001-01-01,0001-01-01
3,0130556,BB,001,EXPIRED,2000-02-14,2006-05-17,2006-05-17,2007-02-11,2006-05-17
4,0130556,BC,001,EXPIRED,2009-10-23,2010-09-19,2010-09-19,0001-01-01,0001-01-01
5,0130556,BC,002,EXPIRED,2010-03-02,2010-11-28,2010-11-28,0001-01-01,0001-01-01
6,0130556,BC,003,EXPIRED,2010-11-28,2011-08-15,2011-08-15,0001-01-01,0001-01-01
7,0130556,BC,004,EXPIRED,2011-02-07,2011-12-01,2011-12-01,0001-01-01,0001-01-01
8,0130556,BD,001,ACTIVE,2015-07-16,2026-07-22,2026-07-22,0001-01-01,0001-01-01
9,0130556,BD,002,EAR.TERM,2026-07-22,2016-06-27,2035-06-15,0001-01-01,0001-01-01


## Key Takeaways and Variables from Sentence Computation

- This table is NOT unique on offender + commitment prefix - each sentence component has its own "sentence end date"
- Theres also a flag for whether that sentence component has expired or not
- I think we can use "Actual sentence end date" and take the max within a commitment prefix to figure out when an individual was done with that specific sentence
- This is an "inmate" table - it's only computing release dates / sentence end dates for sentences that resulted in incarceration, not those resulting in probation.
- There was something unique about this case (see blog post) which is why we see "ear. term" and "future" as codes in inmate computation status flag. I dont think this is very common, and I think we can see use max of "actual sentence end date"?
- The problem with this is that while we can get a sentence begin date for sentences that resulted in probation, we can't get a sentence end date. Eliminating sentences that resulted in probation would mean limiting our definition of recidivism or re-incarceration, NOT re-conviction. I'm okay with this, and its a valid definition of recidivism, but wanted to flag this. Thoughts?



### Putting it all together

If I've understood the above tables and variables correctly, I think here's some pseudocode for the query we will want to write to generate our recidivism flag:

- If we agree to define recidivism and reincarceration, we first limit court commitment data to where "new period of incarceration flag" equals Y. Other variables to keep: offender DOC number, commitment prefix, earliest sentence effective date, and most serious offense code

- We join this to (a version of) sentence component to get "most serious offense code" where missing (resulting dataset A)

- We make sentence computation unique on Offender DOC Number and Commitment prefix by taking the maximum of the "actual sentence end date" (resulting dataset B)

- We join A and B on offender DOC number and commitment prefix so we have the following:
    - Offender DOC
    - Commitment Prefix
    - Sentence begin date
    - Sentence end date
    - Most serious offense for that sentence
    
- From here, we apply our rules of recidivism, something along the lines of:
    - If offender has multiple commitment prefixes, we will need to compare sentence end date of one with sentence begin date of the next one. If this difference is less than or equal to XXX (e.g. 3 years), then the first sentence gets a recidivism flag = 1, and recidivism_public_safety_risk_flag = X based on the offense of the SECOND sentence
    - If the difference is greater than XXX, then recidivism = 0 for that sentence
    - If there is only one commitment prefix (i.e. one sentence) that has ended, then recidivism = 0
    - At the end, drop currently active sentences because we can't compute recidivism or not for those



### After all this, we join with other tables to bring in demographic / other features

IGNORE EVERYTHING BELOW THIS I WAS JUST MESSING AROUND

In [85]:

df = qd.query_db(conn)
conn.close()

In [86]:
df

,OFFENDER_NC_DOC_ID_NUMBER,COMMITMENT_PREFIX,SENTENCE_COMPONENT_NUMBER,COUNTY_OF_CONVICTION_CODE,COURT_DOCKET_NUMBER,PUNISHMENT_TYPE_CODE,COURT_TYPE_CODE,COMPONENT_DISPOSITION_CODE,CMP_DISPOSITION_CODE_2_OF_2,NUMBER_OF_COUNTS,TYPE_OF_COUNT_CODE,PRIMARY_OFFENSE_CODE,OFFENSE_QUALIFIER_CODE,DATE_OFFENSE_COMMITTED__BEGIN,DATE_OFFENSE_COMMITTED__END,NC_GENERAL_STATUTE_NUMBER,PRIMARY_FELONYMISDEMEANOR_CD,SENTENCING_PENALTY_CLASS_CODE,PRIOR_RCD_POINTSCONVICTIONS,PRIOR_RECORD_LEVEL_CODE,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,LENGTH_OF_SUPERVISION,SUPERVISION_TERM_EXTENSION,SUPERVISION_TO_FOLLOW_INCAR,SPLIT_SENTENCE_ACTIVE_TERM,GS_MAXIMUM_SENTENCE_ALLOWED,SERVING_MIN_OR_MAX_TERM_CODE,SENTENCE_TYPE_CODE,SENTENCE_TYPE_CODE1,SENTENCE_TYPE_CODE2,SENTENCE_TYPE_CODE3,SENTENCE_TYPE_CODE4,SENTENCE_TYPE_CODE5,CREDITS_FOR_JAIL_DAYS_SERVED,ICC_JAIL_CREDITS_IN_DAYS,SENTENCE_CHAINING_TYPE_CODE,PRIOR_COMMITMENT_PREFIX,PRIOR_COMPONENT_IDENTIFIER,PP_SUPVTERM_CHAIN_TYPE_CODE,PP_PRIOR_COMMITMENT_PREFIX,PRIOR_PP_COMMNTCOMPONENT_ID,TIME_COMPUTATION_EXCEPTION_CD,SENTENCE_CONVICTION_DATE,SENTENCE_EFFECTIVEBEGIN_DATE,DELEGATED_AUTHORITY_FLAG,INMATE_SENTENCE_STATUS_CODE,INMATE_COMPONENT_STATUS_DATE,PP_CASE_STATUS,PP_COMPONENT_STATUS_DATE,DATE_OF_LAST_UPDATE,TIME_OF_LAST_UPDATE,ORIGINAL_DATA_ENTRY_DATE,ORIGINAL_SENTENCE_AUDIT_CODE,DATE_OF_LAST_UPDATE_TWO,TIME_OF_LAST_UPDATE_TWO
0,0130556,01,001,NEW HANOVER,92020074,PRE-SS (FAIR) DCC,DISTRICT,GUILTY,GUILTY,001,None,DRUG PARA - USE/POSSESS,PRINCIPAL,1992-10-09,1992-10-09,None,MISD.,MISD.(PRE-STRUCTURE),000,None,0010000,0000000,None,None,None,0000000,0000000,None,PROBATION,SUSPENDED SENTENCE,COUNTY JAIL,None,None,None,00000,00000,None,None,None,INITIAL,None,None,NOT APPLICABLE,1992-10-23,1992-10-23,None,None,0001-01-01,ELECT TO SERVE SERVE,1995-02-07,0001-01-01,01:00:00,0001-01-01,None,0001-01-01,01:00:00
1,0130556,02,001,NEW HANOVER,94021895,PRE-SS (FAIR) DCC,SUPERIOR,GUILTY,GUILTY,001,None,LARCENY (OVER $200),PRINCIPAL,1994-07-11,1994-07-11,None,FELON,CLASS H,000,None,0030000,0000000,None,None,None,0000000,0000000,None,PROBATION,SUSPENDED SENTENCE,DEPT OF CORR DIV OF PRISONS,None,None,None,00000,00000,None,None,None,INITIAL,None,None,NOT APPLICABLE,1995-04-06,1995-04-06,None,None,0001-01-01,ELECT TO SERVE SERVE,1996-01-11,1996-01-17,10:04:55,0001-01-01,None,0001-01-01,01:00:00
2,0130556,03,001,MECKLENBURG,97020381,COMMUNITY SS (DCC),DISTRICT,GUILTY,None,001,CONCURRENT,DRIV LICENSE REVOKED,PRINCIPAL,1997-05-29,1997-05-29,20-28,MISD.,CLASS 1 MISDEMEANOR SS,000,LEVEL II,0000115,0000115,None,None,None,000 0 0,None,MAX.TERM:,PROBATION,SUSPENDED SENTENCE,COUNTY JAIL,None,None,None,00000,00000,None,None,None,INITIAL,None,None,NOT APPLICABLE,1997-08-01,1997-08-01,Y,None,0001-01-01,REVOKED REVOK,1998-07-08,1998-07-28,10:36:40,1997-08-27,None,1997-08-27,14:29:17
3,0130556,04,001,MECKLENBURG,97040899,COMMUNITY SS (DCC),SUPERIOR,GUILTY,None,001,CONCURRENT,USE SMOKE SCREEN,PRINCIPAL,1997-10-24,1997-10-24,90-95,FELON,CLASS I,003,LEVEL II,0000600,0000800,None,None,None,000 0 0,None,MAX.TERM:,PROBATION,None,None,None,None,None,00000,00000,None,None,None,CONCURRENT,03,001,NOT APPLICABLE,1998-05-01,1998-05-01,N,None,0001-01-01,REVOKED REVOK,1998-07-08,1998-07-28,10:36:40,1998-05-06,None,1998-05-06,07:41:56
4,0130556,AA,001,NEW HANOVER,92020074,FAIR MISDEMEAN,DISTRICT,UNKNOWN,None,001,None,DRUG PARA - USE/POSSESS,PRINCIPAL,0001-01-01,0001-01-01,None,MISD.,MISD.(PRE-STRUCTURE),000,None,0000000,0010000,0000000,0000000,0000502,0000000,0000000,MAX.TERM:,DEPT OF CORR DIV OF PRISONS,PROBATION REVOCATION,None,None,None,None,00001,00000,INITIAL,None,None,None,None,None,NOT APPLICABLE,1992-10-23,1995-02-07,None,ACTIVE,1995-02-10,None,0001-01-01,0001-01-01,01:00:00,0001-01-01,CONVERSION CS,0001-01-01,01:00:00
5,0130556,BA,001,NEW HANOVER,94021895,FAIR FELONS,SUPERIOR,UNKNOWN,None,001,CONCURRENT,LARCENY (OVER $200),PRINCIPAL,1994-07-11,1994-07-11,14-72,FELON,CLASS H,000,None,0000000,0030000,None,None,None,0000000,0000000,MAX.TERM:,

In [83]:
df

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,INMATE_SENTENCE_COMPONENT,INMATE_COMPUTATION_STATUS_FLAG,SENTENCE_BEGIN_DATE_FOR_MAX,ACTUAL_SENTENCE_END_DATE,PROJECTED_RELEASE_DATE_PRD,PAROLE_DISCHARGE_DATE,PAROLE_SUPERVISION_BEGIN_DATE
0,0130556,AA,001,EXPIRED,1995-02-07,1995-03-09,1995-08-08,0001-01-01,0001-01-01
1,0130556,BA,001,EXPIRED,1996-01-11,1996-08-01,1997-06-17,1997-04-14,1996-08-01
2,0130556,BA,002,EXPIRED,1996-01-11,1996-07-19,1996-07-19,0001-01-01,0001-01-01
3,0130556,BB,001,EXPIRED,2000-02-14,2006-05-17,2006-05-17,2007-02-11,2006-05-17
4,0130556,BC,001,EXPIRED,2009-10-23,2010-09-19,2010-09-19,0001-01-01,0001-01-01
5,0130556,BC,002,EXPIRED,2010-03-02,2010-11-28,2010-11-28,0001-01-01,0001-01-01
6,0130556,BC,003,EXPIRED,2010-11-28,2011-08-15,2011-08-15,0001-01-01,0001-01-01
7,0130556,BC,004,EXPIRED,2011-02-07,2011-12-01,2011-12-01,0001-01-01,0001-01-01
8,0130556,BD,001,ACTIVE,2015-07-16,2026-07-22,2026-07-22,0001-01-01,0001-01-01
9,0130556,BD,002,EAR.TERM,2026-07-22,2016-06-27,2035-06-15,0001-01-01,0001-01-01


In [80]:
df

,OFFENDER_NC_DOC_ID_NUMBER,COMMITMENT_PREFIX,COMMITTED_LAST_NAME,COMMITTED_FIRST_NAME,COMMITTED_MIDDLE_NAME,COMMITTED_NAME_SUFFIX,OFFENDER_ADMISSIONINTAKE_DATE,PP_CASE_INTAKE_DATE,INMATE_COMMITMENT_STATUS_FLAG,COMMITMENT_STATUS_DATE,EARLIEST_SENTENCE_EFFECTIVE_DT,NEW_PERIOD_OF_INCARCERATION_FL,MOST_SERIOUS_OFFENSE_CODE,CO_OF_CONV_MOST_SERIOUS_OFFNSE,TOTAL_SENTENCE_LENGTH,TOTAL_JAIL_CREDITS_IN_DAYS,NO_RESTITUTION_FLAG,PP_COMMITMENT_STATUS_FLAG,PP_COMMITMENT_STATUS_DATE,TOTAL_LENGTH_OF_SUPERVISION,PED_PRIOR_TO_1995_CONVERSION,DATE_OF_LAST_UPDATE,TIME_OF_LAST_UPDATE,NEW_PERIOD_OF_SUPERVISION_FLAG,TYPE_OF_OLD_PE_DATE_CODE
0,0130556,01,FLEMING,TIMOTHY,CHADWICK,None,0001-01-01,1992-10-23,None,0001-01-01,1992-10-23,None,None,None,None,0000,None,ELECT TO SERVE SERVE,1995-02-07,None,0001-01-01,0001-01-01,01:00:00,None,None
1,0130556,02,FLEMING,TIMOTHY,CHADWICK,None,0001-01-01,1995-04-06,None,0001-01-01,1995-04-06,None,None,None,None,0000,None,ELECT TO SERVE SERVE,1996-01-11,None,0001-01-01,1996-01-17,10:04:55,None,None
2,0130556,03,FLEMING,TIMOTHY,CHADWICK,None,0001-01-01,1997-08-01,None,0001-01-01,1997-08-01,N,DRIV LICENSE REVOKED,MECKLENBURG,None,0000,None,REVOKED REVOK,1998-07-08,None,0001-01-01,1998-07-28,10:36:40,Y,None
3,0130556,04,FLEMING,TIMOTHY,CHADWICK,None,0001-01-01,1997-08-01,None,0001-01-01,1998-05-01,N,USE SMOKE SCREEN,MECKLENBURG,None,0000,None,REVOKED REVOK,1998-07-08,None,0001-01-01,1998-07-28,10:36:40,N,None
4,0130556,AA,FLEMING,TIMOTHY,CHAD,None,1995-02-10,1995-03-09,ACTIVE,1995-02-10,1995-02-07,Y,None,None,None,None,None,NORMAL NORM,1995-03-09,None,1995-02-06,0001-01-01,01:00:00,None,REG.PAROLE
5,0130556,BA,FLEMING,TIMOTHY,CHAD,None,1996-04-18,1996-08-01,ACTIVE,1996-04-18,1996-01-11,Y,LARCENY (OVER $200),NEW HANOVER,None,0000,None,NORMAL NORM,1996-08-01,None,0001-01-01,1996-08-22,15:21:08,N,None
6,0130556,BB,FLEMING,TIMOTHY,CHAD,None,2000-03-02,2006-05-17,ACTIVE,2000-03-02,2000-02-14,Y,HABITUAL FELON,MECKLENBURG,None,0151,None,NORMAL NORM,2006-05-17,None,0001-01-01,2006-05-17,17:13:57,Y,None
7,0130556,BC,FLEMING,TIMOTHY,CHAD,None,2009-11-19,0001-01-01,ACTIVE,2009-11-19,2009-10-23,Y,POSSESS WITS SCHEDULE I,MECKLENBURG,0010400,0098,None,NORMAL NORM,0001-01-01,0000000,0001-01-01,2011-03-08,14:38:36,N,None
8,0130556,BD,FLEMING,TIMOTHY,CHADWICK,None,2015-07-22,0001-01-01,ACTIVE,2015-07-22,2015-07-16,Y,HABITUAL FELON,MECKLENBURG,0280125,0774,None,NORMAL NORM,0001-01-01,0000000,0001-01-01,2016-07-14,09:23:57,N,None


In [5]:
df['SENTENCE_CONVICTION_DATE'].max()

'2020-01-08'

In [6]:
df[['PRIMARY_OFFENSE_CODE','SENTENCING_PENALTY_CLASS_CODE']]

,PRIMARY_OFFENSE_CODE,SENTENCING_PENALTY_CLASS_CODE
0,LARCENY (OVER $200),MISD.(PRE-STRUCTURE)
1,POSSESS WITS SCHEDULE II,CLASS H
2,DWI LEVEL 2,NON CLASS CODE
3,SELL SCHEDULE II,CLASS H
4,SELL SCHEDULE II,CLASS H
...,...,...
995,FORGERY,CLASS I
996,KEEPING HIRED PROPERTY,MISD.(PRE-STRUCTURE)
997,WORTHLESS CHECK,CLASS 2 MISDEMEANOR SS
998,WORTHLESS CHECK,CLASS 2 MISDEMEANOR SS


In [7]:
df.columns

Index(['OFFENDER_NC_DOC_ID_NUMBER', 'COMMITMENT_PREFIX',
       'SENTENCE_COMPONENT_NUMBER', 'COUNTY_OF_CONVICTION_CODE',
       'COURT_DOCKET_NUMBER', 'PUNISHMENT_TYPE_CODE', 'COURT_TYPE_CODE',
       'COMPONENT_DISPOSITION_CODE', 'CMP_DISPOSITION_CODE_2_OF_2',
       'NUMBER_OF_COUNTS', 'TYPE_OF_COUNT_CODE', 'PRIMARY_OFFENSE_CODE',
       'OFFENSE_QUALIFIER_CODE', 'DATE_OFFENSE_COMMITTED__BEGIN',
       'DATE_OFFENSE_COMMITTED__END', 'NC_GENERAL_STATUTE_NUMBER',
       'PRIMARY_FELONYMISDEMEANOR_CD', 'SENTENCING_PENALTY_CLASS_CODE',
       'PRIOR_RCD_POINTSCONVICTIONS', 'PRIOR_RECORD_LEVEL_CODE',
       'MINIMUM_SENTENCE_LENGTH', 'MAXIMUM_SENTENCE_LENGTH',
       'LENGTH_OF_SUPERVISION', 'SUPERVISION_TERM_EXTENSION',
       'SUPERVISION_TO_FOLLOW_INCAR', 'SPLIT_SENTENCE_ACTIVE_TERM',
       'GS_MAXIMUM_SENTENCE_ALLOWED', 'SERVING_MIN_OR_MAX_TERM_CODE',
       'SENTENCE_TYPE_CODE', 'SENTENCE_TYPE_CODE1', 'SENTENCE_TYPE_CODE2',
       'SENTENCE_TYPE_CODE3', 'SENTENCE_TYPE_CODE4', 'SENT

In [65]:
df.groupby('INMATE_COMPUTATION_STATUS_FLAG').count()

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,INMATE_SENTENCE_COMPONENT,SENTENCE_BEGIN_DATE_FOR_MAX,ACTUAL_SENTENCE_END_DATE,PROJECTED_RELEASE_DATE_PRD,PAROLE_DISCHARGE_DATE,PAROLE_SUPERVISION_BEGIN_DATE
INMATE_COMPUTATION_STATUS_FLAG,,,,,,,,
ACTIVE,11,11,11,11,11,11,11,11
EXPIRED,978,978,978,978,978,978,978,978
FUTURE,9,9,9,9,9,9,9,9
PAROLED,2,2,2,2,2,2,2,2


In [49]:
df[df['OFFENDER_NC_DOC_ID_NUMBER']=='0002456']

,OFFENDER_NC_DOC_ID_NUMBER,COMMITMENT_PREFIX,COMMITTED_LAST_NAME,COMMITTED_FIRST_NAME,COMMITTED_MIDDLE_NAME,COMMITTED_NAME_SUFFIX,OFFENDER_ADMISSIONINTAKE_DATE,PP_CASE_INTAKE_DATE,INMATE_COMMITMENT_STATUS_FLAG,COMMITMENT_STATUS_DATE,EARLIEST_SENTENCE_EFFECTIVE_DT,NEW_PERIOD_OF_INCARCERATION_FL,MOST_SERIOUS_OFFENSE_CODE,CO_OF_CONV_MOST_SERIOUS_OFFNSE,TOTAL_SENTENCE_LENGTH,TOTAL_JAIL_CREDITS_IN_DAYS,NO_RESTITUTION_FLAG,PP_COMMITMENT_STATUS_FLAG,PP_COMMITMENT_STATUS_DATE,TOTAL_LENGTH_OF_SUPERVISION,PED_PRIOR_TO_1995_CONVERSION,DATE_OF_LAST_UPDATE,TIME_OF_LAST_UPDATE,NEW_PERIOD_OF_SUPERVISION_FLAG,TYPE_OF_OLD_PE_DATE_CODE


In [ ]:
# it seems like commitment prefixes that are numbers correspond to sentence type code that is probation where as
# commitment prefixes that are letters correspond to sentence_type_code = DEPT OF CORR DIV OF PRISONS 




In [ ]:
# for the purposes of our project, we probably only want to look at the universe of sentences where the individual shows up 
# in the inmate profile dataset, i.e. because they are or have been incarcerated. sentence component will include many that are
# on probation
